In [1]:
    import requests
    import pandas as pd
    from bs4 import BeautifulSoup
    from io import StringIO
    import time

    def get_day_price(code, max_pages=599):
        base_url = f"https://finance.naver.com/item/sise_day.naver?code={code}"
        headers = {"User-Agent": "Mozilla/5.0"}
        df_total = pd.DataFrame()

        for page in range(1, max_pages + 1):
            url = f"{base_url}&page={page}"
            res = requests.get(url, headers=headers)
            if res.status_code != 200:
                print(f"페이지 {page} 요청 실패: 상태 코드 {res.status_code}")
                break

            soup = BeautifulSoup(res.text, "html.parser")
            table = soup.find("table", class_="type2")
            if table is None:
                print(f"페이지 {page}에서 테이블을 찾을 수 없습니다.")
                break

            df = pd.read_html(StringIO(str(table)), header=0)[0]
            df = df.dropna()

            # 👉 만약 페이지에 데이터가 없는 경우 자동 종료
            if df.empty:
                print(f"\n✅ 데이터 없음: 페이지 {page}에서 종료합니다.")
                break

            # 전일비 처리
            def parse_change(val):
                val = str(val).strip().replace(",", "")
                if val.startswith("상승"):
                    return int(val.replace("상승", "").strip())
                elif val.startswith("하락"):
                    return -int(val.replace("하락", "").strip())
                elif val.startswith("보합"):
                    return 0
                else:
                    return None

            df["전일비"] = df["전일비"].apply(parse_change)

            # 거래량 처리
            df["거래량"] = df["거래량"].astype(str).str.replace(",", "")
            df["거래량"] = df["거래량"].apply(lambda x: int(float(x)))

            # ✅ 출력 – 첫 페이지만 헤더 포함
            if page == 1:
                print(df.to_string(index=False))  # 헤더 포함
            else:
                print(df.to_string(index=False, header=False))  # 헤더 생략

            df_total = pd.concat([df_total, df], ignore_index=True)
            time.sleep(0.5)

        return df_total

    # 종목 코드 및 기업명 설정
    code = "042660"
    company_name = "한화오션"

    # 데이터 수집
    df = get_day_price(code, max_pages=599)

    # 컬럼명 정리
    df.columns = ["날짜", "종가", "전일비", "시가", "고가", "저가", "거래량"]

    # 날짜 형식 변환
    df["날짜"] = pd.to_datetime(df["날짜"])

    # ✅ 날짜 기준으로 중복 제거
    df = df.drop_duplicates(subset="날짜", keep="first")

    # 날짜 내림차순 정렬
    df = df.sort_values("날짜", ascending=False)

    # CSV 파일로 저장
    filename = f"{company_name}.csv"
    df.to_csv(filename, index=False, encoding="utf-8-sig")
    print(f"\n✅ {filename} 파일이 저장되었습니다.")


        날짜      종가   전일비      시가      고가      저가     거래량
2025.04.25 89900.0  9000 82700.0 90000.0 81900.0 7364955
2025.04.24 80900.0  1000 80500.0 82400.0 79400.0 2616604
2025.04.23 79900.0   900 80000.0 81300.0 78800.0 2700319
2025.04.22 79000.0  1100 76900.0 79600.0 76800.0 2051026
2025.04.21 77900.0  -500 79200.0 79200.0 76600.0 1369990
2025.04.18 78400.0   500 78700.0 79800.0 76900.0 2214342
2025.04.17 77900.0   600 77700.0 78600.0 76500.0 1479940
2025.04.16 77300.0 -2100 77200.0 79400.0 76200.0 2023987
2025.04.15 79400.0  -700 79900.0 80000.0 78500.0 1973213
2025.04.14 80100.0  2500 81100.0 81100.0 78800.0 2553658
2025.04.11 77600.0  4500 75000.0 78200.0 74100.0 4571802
2025.04.10 73100.0  4500 74500.0 74600.0 71900.0 2764273
2025.04.09 68600.0   900 70900.0 70900.0 67500.0 2926001
2025.04.08 67700.0  5200 65600.0 68000.0 65000.0 2668105
2025.04.07 62500.0 -6800 66000.0 66100.0 62350.0 3000005
2025.04.04 69300.0 -2300 71200.0 72300.0 66900.0 3616908
2025.04.03 71600.0  2000 68600.